In [1]:
import os
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import requests


from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_predict, train_test_split
from sklearn.metrics import (roc_auc_score, confusion_matrix, precision_score,
                             recall_score, roc_curve, precision_recall_curve,auc)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score
import lightgbm as lgb

# from utils import plot_confusion_matrix

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [35]:
window_size = 4
# размер окна
window_method = 1
# 0 - если хоть один элемент аномальный то окно аномально, 1 - если все, 2 если больше половины
experiment_message = 'тестируем лес'
file_delimeter = 4
bad_vals_len=960

In [36]:
r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+'start')
dir_path = '/Users/alisa95/Downloads/FEATURES-2014-2015/'
# names_list = sorted(os.listdir(dir_path)[::4])
names_list = ['part-004.csv', 'part-007.csv', 'part-011.csv', 'part-012.csv', 'part-015.csv', 'part-018.csv', 'part-021.csv', 'part-028.csv', 'part-033.csv', 'part-036.csv', 'part-042.csv', 'part-047.csv', 'part-055.csv', 'part-059.csv', 'part-066.csv', 'part-073.csv', 'part-074.csv', 'part-078.csv', 'part-082.csv', 'part-087.csv', 'part-088.csv', 'part-091.csv', 'part-092.csv', 'part-094.csv', 'part-097.csv', 'part-101.csv', 'part-105.csv', 'part-106.csv', 'part-108.csv', 'part-114.csv', 'part-116.csv', 'part-131.csv', 'part-133.csv', 'part-136.csv', 'part-138.csv', 'part-141.csv', 'part-149.csv', 'part-150.csv', 'part-152.csv', 'part-154.csv', 'part-157.csv', 'part-163.csv', 'part-164.csv', 'part-176.csv', 'part-179.csv', 'part-181.csv', 'part-183.csv', 'part-184.csv', 'part-186.csv', 'part-199.csv']
print(names_list)
sequences = pd.concat([pd.read_csv(dir_path + name, parse_dates=['timestamp'])for name in names_list], ignore_index=True)

r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+'read data done')

['part-004.csv', 'part-007.csv', 'part-011.csv', 'part-012.csv', 'part-015.csv', 'part-018.csv', 'part-021.csv', 'part-028.csv', 'part-033.csv', 'part-036.csv', 'part-042.csv', 'part-047.csv', 'part-055.csv', 'part-059.csv', 'part-066.csv', 'part-073.csv', 'part-074.csv', 'part-078.csv', 'part-082.csv', 'part-087.csv', 'part-088.csv', 'part-091.csv', 'part-092.csv', 'part-094.csv', 'part-097.csv', 'part-101.csv', 'part-105.csv', 'part-106.csv', 'part-108.csv', 'part-114.csv', 'part-116.csv', 'part-131.csv', 'part-133.csv', 'part-136.csv', 'part-138.csv', 'part-141.csv', 'part-149.csv', 'part-150.csv', 'part-152.csv', 'part-154.csv', 'part-157.csv', 'part-163.csv', 'part-164.csv', 'part-176.csv', 'part-179.csv', 'part-181.csv', 'part-183.csv', 'part-184.csv', 'part-186.csv', 'part-199.csv']


In [37]:
# сортировка
sequences.sort_values(['host', 'process', 'timestamp'], ascending=True, inplace=True)

In [38]:
# на выходе маска new_mask для того что бы проставить лишних bad_vals_len тру перед тру

mask = sequences['isAnomaly'].values.astype(bool)
new_mask = sequences['isAnomaly'].values.astype(bool)
print([len(list(group)) for key, group in itertools.groupby(new_mask) if key])
print([len(list(group)) for key, group in itertools.groupby(~new_mask) if key])
i=0
for index in range(len(new_mask)-2,0 ,-1):
    if(new_mask[index]==False and new_mask[index+1]==True and i<bad_vals_len):
        new_mask[index]=True
        i=i+1
    else:
        i=0

# print([len(list(group)) for key, group in itertools.groupby(new_mask) if key])
# print([len(list(group)) for key, group in itertools.groupby(~new_mask) if key])

new_mask = new_mask ^ mask

# print([len(list(group)) for key, group in itertools.groupby(new_mask) if key])
# print([len(list(group)) for key, group in itertools.groupby(~new_mask) if key])

sequences = sequences[~new_mask]

len(mask)
r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+'bad_vals_droped')

[23, 650, 292, 132, 34, 1840, 1723, 131, 10247, 176, 6834, 451, 1256, 3234, 94, 256, 2673, 23, 4313, 493, 52, 10286, 1598, 145, 1722, 34]
[427172, 96054, 16647, 185951, 68, 9801, 34110, 50240, 56970, 48198, 61417, 34311, 8454, 18719, 57557, 80344, 2810, 21274, 19129, 48059, 108401, 13222, 85038, 154919, 149459, 48774, 36006]


In [39]:
labels = sequences['isAnomaly']
sequences = sequences.drop(columns=['isAnomaly', 'host', 'process', 'timestamp'])

In [ ]:
window_sequences = pd.DataFrame()
window_labels = pd.DataFrame()
for i in range(window_size):
    window_sequences = pd.concat([window_sequences, sequences.shift(periods=i)], axis=1)
    window_labels[f'shifted_{i}'] = labels.shift(periods=i)

sequences = window_sequences
labels = window_labels

sequences.dropna(inplace=True)
labels.dropna(inplace=True)

num_columns = labels.shape[1]
sum_lables = labels.sum(axis=1)
if(window_method==0):
    for val_index in sum_lables.index:
        if sum_lables[val_index] != 0:
            sum_lables[val_index] = 1
elif(window_method==1):
    for val_index in sum_lables.index:
        if sum_lables[val_index] == num_columns:
            sum_lables[val_index] = 1
        else:
            sum_lables[val_index] = 0
            
labels=sum_lables
r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+'window reshape done')

In [ ]:
# для окна 1 max_depth=5
try:
    forest = RandomForestClassifier(n_estimators=200, max_depth=4, n_jobs=-1, random_state=1488)
#     forest = LGBMClassifier(max_depth=5, n_estimators=80, n_jobs=-1, learning_rate=0.05, random_state=1488, num_leaves=60)
    forest_labels = cross_val_predict(forest, sequences, labels,n_jobs=-1, cv=4, method='predict_proba')[:, 1]
    experiment_message = 'RandomForestClassifier(n_estimators=200, max_depth=None, n_jobs=-1, random_state=1488, max_leaf_nodes=5)'
except Exception as e:
    message='!!! EXCEPTION !!!'
    print(str(e))
    r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+message)

In [ ]:
# precision recall
print("precision_score: ",precision_score(labels, forest_labels > 0.02), "\nrecall_score: ",recall_score(labels, forest_labels > 0.02))
# roc_auc_score
print("roc_auc_score: ",roc_auc_score(labels, forest_labels))
# roc_curve
fpr, tpr, _ = roc_curve(labels, forest_labels)
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

In [ ]:
# prc_curve
pr, rc, _ = precision_recall_curve(labels, forest_labels)
plt.plot(rc, pr)
plt.xlabel('rc')
plt.ylabel('pr')
print("prc_auc_score: ",auc(rc, pr))

In [ ]:
f2_score = fbeta_score(labels.astype(int), forest_labels.astype(int), beta=2)

In [ ]:
# RandomForest
message='RandomForest done \n  bad_vals_len '+ str(bad_vals_len)+'\n  window_size '+str(window_size)+ ' \n  window_method '+ str(window_method)+'\n  auc_roc '+str(roc_auc_score(labels, forest_labels)) +'\n  auc_rpc '+str(auc(rc, pr))+'\n  f2_score '+str(f2_score)+'\n'+experiment_message

r = requests.get('http://alarmerbot.ru/?key=c51ae3-7b05b8-ee307f&message='+str(message))

In [ ]:
for i in np.arange(0.00, 0.2, 0.0001):
    print('threshold:', i)
    x = confusion_matrix(labels, forest_labels < i).astype(float)
    print(x)
    x /= x.sum(axis=1)[:, None]
    print(x)
    print("pr: ",precision_score(labels, forest_labels < i).astype(float)," ; rc: " ,recall_score(labels, forest_labels < i).astype(float))
    print('\n')

In [ ]:
%%time
scaler = StandardScaler(copy=False)
logreg = LogisticRegression(solver='lbfgs', n_jobs=1)
logreg_pipe = Pipeline([('scaler', scaler), ('logreg', logreg)])
logreg_labels = cross_val_predict(logreg_pipe, sequences, labels,
                                  cv=4, method='predict_proba')[:, 1]

In [ ]:
%%time
boosting = LGBMClassifier(max_depth=7, n_estimators=80, n_jobs=-1)
boosting_labels = cross_val_predict(boosting, sequences, labels, cv=4,
                                    method='predict_proba')[:, 1]

In [ ]:
confusion_matrix(labels, forest_labels)

In [ ]:
precision_score(labels, forest_labels), recall_score(labels, forest_labels)

In [ ]:
confusion_matrix(labels, boosting_labels)

In [ ]:
precision_score(labels, boosting_labels), recall_score(labels, boosting_labels)

In [ ]:
confusion_matrix(labels, logreg_labels)

In [ ]:
precision_score(labels, logreg_labels), recall_score(labels, logreg_labels)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), forest_labels.astype(int),
    classes=class_names, normalize=True, title="Forest Confusion Matrix")
plt.tight_layout()
plt.savefig("images/forest_confusion_matrix.png", dpi=140)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), boosting_labels.astype(int),
    classes=class_names, normalize=True, title="Boosting Confusion Matrix")
plt.tight_layout()
plt.savefig("images/boosting_confusion_matrix.png", dpi=140)

In [ ]:
class_names = np.array(["Healthy", "Anomaly"])
np.set_printoptions(precision=2)
plot_confusion_matrix(labels.astype(int), logreg_labels.astype(int),
    classes=class_names, normalize=True, title="Logreg Confusion Matrix")
plt.tight_layout()
plt.savefig("images/logreg_confusion_matrix.png", dpi=140)

In [ ]:
sequences = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9],[10, 11, 12],[13, 14, 15]]),columns=['a', 'b', 'c'])
labels = pd.Series(np.array([0,0,1,1,0]))

In [ ]:
%%time
forest = RandomForestClassifier(n_estimators=60, n_jobs=-1, random_state=1488)
forest_labels = cross_val_predict(forest, sequences, labels, cv=4,
                                  method='predict_proba')[:, 1]